# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import math

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
from math import sin, cos, sqrt, atan2, radians

In [2]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"
SHP = gpd.read_file("/Users/wesley/Desktop/Columbia_2022_fall/IEOR4501/Final Project/taxi_zones.shp")

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [46]:
def SHP_prepare(SHP):
    SHP['PULocationID'] = SHP['LocationID']
    SHP['DOLocationID'] = SHP['LocationID']
    SHP['pick_lon'] = SHP['geometry'].to_crs(4326).centroid.x
    SHP['pick_lat'] = SHP['geometry'].to_crs(4326).centroid.y
    SHP['drop_lon'] = SHP['geometry'].to_crs(4326).centroid.x
    SHP['drop_lat'] = SHP['geometry'].to_crs(4326).centroid.y
    return SHP

SHP_trans = SHP_prepare(SHP)
SHP_trans

/var/folders/_s/18nnk14x26xd0hw8_j7261xw0000gn/T/ipykernel_24382/1150329011.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  SHP['pick_lon'] = SHP['geometry'].to_crs(4326).centroid.x
/var/folders/_s/18nnk14x26xd0hw8_j7261xw0000gn/T/ipykernel_24382/1150329011.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  SHP['pick_lat'] = SHP['geometry'].to_crs(4326).centroid.y
/var/folders/_s/18nnk14x26xd0hw8_j7261xw0000gn/T/ipykernel_24382/1150329011.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  SHP['drop_lon'] = SHP['geometry'].to_crs(4326).centroid.x
/var/folders/_s/18n

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,pick_lon,pick_lat,drop_lon,drop_lat,PULocationID,DOLocationID
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19...",-74.174000,40.691831,-74.174000,40.691831,1,1
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343...",-73.831299,40.616745,-73.831299,40.616745,2,2
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ...",-73.847422,40.864474,-73.847422,40.864474,3,3
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20...",-73.976968,40.723752,-73.976968,40.723752,4,4
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14...",-74.188484,40.552659,-74.188484,40.552659,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1025414.782 270986.139, 1025138.624 ...",-73.852215,40.897932,-73.852215,40.897932,259,259
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011466.966 216463.005, 1011545.889 ...",-73.906306,40.744235,-73.906306,40.744235,260,260
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980555.204 196138.486, 980570.792 19...",-74.013023,40.709139,-74.013023,40.709139,261,261
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999804.795 224498.527, 999824....",-73.946510,40.775932,-73.946510,40.775932,262,262


In [57]:
#get the coordinates if there is only locationID and append the coordinate column to the dataframe
def get_coord(df, SHP):

    #drop the records with null values in locationID
    try: 
        df.dropna(axis=0, how= 'any', subset = ['PULocationID', 'DOLocationID'], inplace=True)
    except:
        pass

    if ('PULocationID' in df.columns) and ('DOLocationID' in df.columns): #if only the zone ID is given
        df = pd.merge(df, SHP[['PULocationID', 'pick_lon', 'pick_lat']], left_on = 'PULocationID', right_on = 'PULocationID', how = 'left')
        df = pd.merge(df, SHP[['DOLocationID', 'drop_lon', 'drop_lat']], left_on = 'DOLocationID', right_on = 'DOLocationID', how = 'left')
        
    df.dropna(axis=0, how= 'any', subset = ['pick_lon', 'pick_lat'], inplace=True)
    df = df.loc[(df['pick_lon'] <= -73.717047) & (df['pick_lon'] >= -74.242330) & (df['drop_lon'] <= -73.717047) & (df['drop_lon'] >= -74.242330) 
            & (df['pick_lat'] >= 40.560445) & (df['pick_lat'] <= 40.908524) & (df['drop_lat'] >= 40.560445) & (df['drop_lat'] <= 40.908524)]
    return df
         


taxi_data_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-01.parquet"

response = requests.get(taxi_data_url, stream=True)
with open("lecture_08_taxi_data.parquet", "wb") as f:
    for chunk in response.iter_content(chunk_size=1024): 
        if chunk:
            f.write(chunk)
data = pd.read_parquet('lecture_08_taxi_data.parquet', engine='pyarrow')

df = get_coord(data, SHP_trans)




In [59]:
def calculate_distance(pick_lon, pick_lat, drop_lon, drop_lat):
    R = 6373.0 #approximate earth radius

    lat1 = radians(pick_lat)
    lon1 = radians(pick_lon)
    lat2 = radians(drop_lat)
    lon2 = radians(drop_lon)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    #formula for calculating the distance (km)
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

    

In [60]:
def add_distance_column(df):
    df['distance'] = df[["pick_lon", "pick_lat", "drop_lon", "drop_lat"]].apply(lambda x: calculate_distance(*x), axis = 1)
    return df

add_distance_column(df)   

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pick_lon,pick_lat,drop_lon,drop_lat,distance
0,2,2014-01-01 00:02:00,2014-01-01 00:04:00,6,0.0,1,None,146,146,1,...,0.0,0.0,4.52,NaN,None,-73.934829,40.754243,-73.934829,40.754243,0.000000
1,2,2014-01-01 00:06:00,2014-01-01 00:09:00,5,0.0,1,None,146,146,1,...,0.0,0.0,4.55,NaN,None,-73.934829,40.754243,-73.934829,40.754243,0.000000
2,2,2014-01-01 00:10:00,2014-01-01 00:13:00,5,0.0,1,None,146,146,1,...,0.0,0.0,4.58,NaN,None,-73.934829,40.754243,-73.934829,40.754243,0.000000
4,1,2014-01-01 00:29:18,2014-01-01 00:35:13,2,1.8,1,N,229,262,2,...,0.0,0.0,8.50,NaN,None,-73.965146,40.756729,-73.946510,40.775932,2.650867
5,1,2014-01-01 00:36:33,2014-01-01 00:55:19,3,4.8,1,N,140,7,2,...,0.0,0.0,18.50,NaN,None,-73.954739,40.765484,-73.919694,40.761493,2.985577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13786318,1,2014-01-31 23:22:15,2014-01-31 23:30:56,1,4.0,1,N,148,112,1,...,0.0,0.0,18.20,NaN,None,-73.990896,40.718938,-73.949540,40.729506,3.679001
13786319,1,2014-01-31 23:43:13,2014-02-01 00:12:35,1,4.1,1,N,255,249,1,...,0.0,0.0,22.00,NaN,None,-73.957418,40.718804,-74.002875,40.734576,4.214220
13786320,1,2014-01-31 23:04:12,2014-01-31 23:11:15,1,1.6,1,N,236,151,1,...,0.0,0.0,10.80,NaN,None,-73.957012,40.780436,-73.968168,40.797962,2.163971
13786321,1,2014-01-31 23:19:58,2014-01-31 23:29:45,1,1.6,1,N,143,163,2,...,0.0,0.0,10.00,NaN,None,-73.987646,40.775965,-73.977569,40.764421,1.539240


### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [69]:
def find_taxi_parquet_links():
    #construct the regrex expression list which contains the time in which we would like to analyze (2009-01 to 2015-06)
    regrexlist = []
    for y in range(2009, 2015):
        for m in range(1, 13):
            if m < 10:
                m = '0' + str(m)
            regrexlist.append(str(y) + '-' + str(m))
    for m in range(1, 7):
        regrexlist.append(str(2015) + '-' + '0' + str(m))

    #get all the urls and filter with the regrex above
    TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
    soup = bs4.BeautifulSoup(requests.get(TAXI_URL).content, 'html.parser')
    a = soup.find_all('a')
    res = []
    for ele in a:
        try:
            if ele['title'] == 'Yellow Taxi Trip Records' :
                for reg in regrexlist:
                    if reg in ele['href']:
                        res.append(ele['href'])
        except:
            pass

    return res

find_taxi_parquet_links()

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-06.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-06.pa

In [ ]:
def get_and_clean_month_taxi_data(url):
    raise NotImplemented()

In [ ]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_url:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [78]:

#find the sample size of uber
uber = pd.read_csv("uber_rides_sample.csv")
uber.dropna(axis = 0, how = 'any', inplace = True)
uber['pickup_datetime']
uber['Ym'] = pd.to_datetime(uber['pickup_datetime']).dt.strftime('%Y-%m')
uber.groupby(['Ym']).size()


Ym
2009-01    2555
2009-02    2292
2009-03    2703
2009-04    2574
2009-05    2660
           ... 
2015-02    2225
2015-03    2362
2015-04    2339
2015-05    2449
2015-06    2161
Length: 78, dtype: int64

In [ ]:
def load_and_clean_uber_data():

In [ ]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [11]:
def clean_month_weather_data_hourly(csv_file):
    
    data=pd.read_csv(csv_file)
    
    #由于这是一个聚焦于小时的数据库，删去了所有daily和month的列
    data=data.drop(data.columns[24:112],axis=1)
    
    #有几列只有一个值，没有意义，我全都删了
    data=data.drop(columns=['STATION','LATITUDE','LONGITUDE','ELEVATION','NAME','BackupDirection','BackupDistance','BackupDistanceUnit','BackupElements','BackupElevation','BackupEquipment','BackupLatitude','BackupLongitude','BackupName','WindEquipmentChangeDate'])
    
    #有几列可用的数值实在太少，全都删去
    data=data.drop(columns=['HourlyPrecipitation','HourlyPresentWeatherType','HourlySkyConditions','HourlyStationPressure','HourlyWetBulbTemperature','HourlyPressureChange','HourlyPressureTendency','HourlyWindGustSpeed'])
    

    #对于source这一列来说，只有取4才有意义，删去取0的行，然后将这一列删去
    data=data[data['SOURCE']!="O"]
    data=data.drop(columns=['SOURCE'])
    
    #HourlyWindDirection这一项有一些VRB删去
    data=data[data['HourlyWindDirection']!="VRB"]
    data[['HourlyWindDirection']]=data[['HourlyWindDirection']].astype('float64')
    
    #HourlyDewPointTemperature这一项有一些带s修改
    data['HourlyDewPointTemperature']=data['HourlyDewPointTemperature'].apply( lambda x: str(x)[:-1] if str(x)[-1]=='s' else x)
    data[['HourlyDewPointTemperature']]=data[['HourlyDewPointTemperature']].astype('float64')
    
    #HourlySeaLevelPressure这一项有一些带着s的修改
    data['HourlySeaLevelPressure']=data['HourlySeaLevelPressure'].apply( lambda x: str(x)[:-1] if str(x)[-1]=='s' else x)
    data[['HourlySeaLevelPressure']]=data[['HourlySeaLevelPressure']].astype('float64')
  
    #HourlyDryBulbTemperature这一项有一些带着s的修改
    data['HourlyDryBulbTemperature']=data['HourlyDryBulbTemperature'].apply( lambda x: str(x)[:-1] if str(x)[-1]=='s' else x)
    data[['HourlyDryBulbTemperature']]=data[['HourlyDryBulbTemperature']].astype('float64')

    #HourlyVisibility这一项有一些带着V的要修改，不过这一列在daily里面也没让求取，不知道后续到底要干什么，但先予以保留
    data['HourlyVisibility']=data['HourlyVisibility'].apply( lambda x: str(x)[:-1] if str(x)[-1]=='V' else x)
    data[['HourlyVisibility']]=data[['HourlyVisibility']].astype('float64')

    #删除空缺行
    data=data.dropna(axis=0,how='any')
    
    #重新排列了一下index
    data.reset_index(drop=True, inplace=True)
    
    return data

In [ ]:
def clean_month_weather_data_daily(csv_file):
    data=clean_month_weather_data_hourly(csv_file)
    df=pd.DataFrame()
    
    #提取日期并删除重复项
    df['Date']=data['DATE']
    df['Date']=df['Date'].apply(lambda x : x[0:10])
    df['Date']=df.drop_duplicates()
    df=df.dropna(axis=0,how='any')
    
    #重新排列了一下index
    df.reset_index(drop=True, inplace=True)
    df.set_index(['Date'],inplace=True)


    #计算DailyAverageDewPointTemperature
    data['Daily']=data['DATE'].apply(lambda x : x[0:10])
    data_group=data.groupby('Daily')
    DailyAverageDewPointTemperature=data_group.mean().iloc[:,0]
    df['DailyAverageDewPointTemperature']=DailyAverageDewPointTemperature

    #计算DailyAverageDryBulbTemperature
    DailyAverageDryBulbTemperature=data_group.mean().iloc[:,1]
    df['DailyAverageDryBulbTemperature']=DailyAverageDryBulbTemperature
    
    #计算DailyMaximumDryBulbTemperature
    DailyMaximumDryBulbTemperature=data_group.max().iloc[:,4]
    df['DailyMaximumDryBulbTemperature']=DailyMaximumDryBulbTemperature
    
    #计算DailyMinimumDryBulbTemperature
    DailyMinimumDryBulbTemperature=data_group.min().iloc[:,4]
    df['DailyMinimumDryBulbTemperature']=DailyMinimumDryBulbTemperature

    #计算DailyAverageRelativeHumidity
    DailyAverageRelativeHumidity=data_group.mean().iloc[:,2]
    df['DailyAverageRelativeHumidity']=DailyAverageRelativeHumidity

    #计算DailyAverageSeaLevelPressure
    DailyAverageSeaLevelPressure=data_group.mean().iloc[:,3]
    df['DailyAverageSeaLevelPressure']=DailyAverageSeaLevelPressure

    #计算DailyAverageWindSpeed
    DailyAverageWindSpeed=data_group.mean().iloc[:,6]
    df['DailyAverageWindSpeed']=DailyAverageWindSpeed
    
    #计算DailyPeakWindSpeed
    DailyPeakWindSpeed=data_group.max().iloc[:,9]
    df['DailyPeakWindSpeed']=DailyPeakWindSpeed

    #计算DailyPeakWindDirection 这个我没想出来方法如何计算，如果之后需要再回头考虑，其余变量同理
    
    return df

In [ ]:
def load_and_clean_weather_data(n):
    hourly_dataframes = []
    daily_dataframes = []
    weather_csv_files=[]
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    for i in range(n):
        weather_csv_files.append(r'{year}_weather.csv'.format(year=i+2009))

    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)

    #重新调整一下index 
    hourly_data.reset_index(drop=True, inplace=True)
    daily_data.reset_index(drop=True, inplace=True)

    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [14]:
engine = db.create_engine(DATABASE_URL)

In [15]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [16]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [17]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [18]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [19]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)